In [1]:
import os

from pandas import DataFrame, date_range
import numpy as np

from idealreport.settings import load_settings
from idealreport import create_html
from idealreport.reporter import Reporter
import htmltag

In [2]:
# load settings from settings.hjson
settings = load_settings()

# instantiate reporter object and set output location
output_file = os.path.join(settings['output_path'], 'sample_plots.html')
r = Reporter(title='Sample Plots', output_file=output_file)

# report start
r.h += htmltag.h3('This report provides an overview of the different plot types.')

In [3]:
# vertical bar chart

ch = DataFrame({'Entity': ['Entity 1', 'Entity 2', 'Entity 3', 'Entity 4', 'Entity 5'],
                  'Stat 1': np.random.randn(5).tolist(),
                  'Stat 2': np.random.randn(5).tolist(),
                })

# note the x values are generally assumed to be in the index, with each non index column being a series of y values
# a few exceptions are outlined below
ch = ch.set_index('Entity')
ch = ch.sort_values(by='Stat 1', ascending=False)

# color choices for bars
markers = [
    dict(color = 'rgb(49, 105, 196)'),
    dict(color = 'rgb(195, 197, 201)')
]

# each object can be built by create_html for more flexibility

# graphs are generated from dictionaries with attributes and a data dictionary as an entry
bar_plot = {
    'title': 'Vertical Bar Chart by Create HTML',
    'data': [
        {
            'df': ch,
            'type': 'bar',
            'orientation': 'v',
        },
    ],
    'labelX': 'Entity',
    'labelY': '%',
    'markers' : markers,
    'staticPlot': False
}

r.h += htmltag.h4('Bar Charts')
r.h += create_html.plot(bar_plot)

# all of the remaining examples just use the reporter object to build the dictionaries for create_html
r.plot.bar(df=ch, title='Vertical Bar Chart by Reporter', xlabel='Entity', ylabel='%', markers=markers)

{'data': [{'df':             Stat 1    Stat 2
   Entity                      
   Entity 3  1.367529  0.729625
   Entity 5  0.579619 -0.346069
   Entity 1  0.129179  0.189139
   Entity 2 -0.586665 -3.182817
   Entity 4 -0.662613 -0.039383, 'orientation': 'v', 'type': 'bar'}],
 'markers': [{'color': 'rgb(49, 105, 196)'}, {'color': 'rgb(195, 197, 201)'}],
 'staticPlot': False,
 'title': 'Vertical Bar Chart by Reporter',
 'x': {'label': 'Entity'},
 'y': {'label': '%'}}

In [4]:
# stacked horizontal bar chart, notice additional entry in the data field for orientation

ch = DataFrame({'Entity': ['Entity 1', 'Entity 2', 'Entity 3aaaaaaaaaaaaaaaa', 'Entity 4', 'Entity 5'],
                'Foo': np.random.randn(5).tolist(),
                'Bar': np.random.randn(5).tolist(),
               })
ch = ch.set_index('Entity')
ch = ch[['Foo', 'Bar']] # make sure columns have desired order

markers = [
    dict(color = 'rgb(49, 105, 196)'),
    dict(color = 'rgb(195, 197, 201)')
]

# stacked bars behave oddly with mixed positive / negative numbers, so only use when all numbers are positive or negative within a series
ch = ch.abs()
ch = ch*100

r.plot.barh(df=ch, title='Horizontal Stacked Bar Chart', stacked=True, xlabel='$', markers=markers, layout={'margin':{'l':150}})

{'data': [{'df':                                 Foo         Bar
   Entity                                         
   Entity 1                  48.110578   13.454903
   Entity 2                  85.223416  204.480720
   Entity 3aaaaaaaaaaaaaaaa  40.198631  199.638555
   Entity 4                  14.251279  277.705472
   Entity 5                  66.434562   59.341321,
   'orientation': 'h',
   'type': 'stackedBar'}],
 'layout': {'margin': {'l': 150}},
 'markers': [{'color': 'rgb(49, 105, 196)'}, {'color': 'rgb(195, 197, 201)'}],
 'staticPlot': False,
 'title': 'Horizontal Stacked Bar Chart',
 'x': {'label': '$'}}

In [5]:
# horizontal overlay bar chart, notice additional entry in the data field for orientation

ch = DataFrame({'Entity': ['Entity 1', 'Entity 2', 'Entity 3', 'Entity 4', 'Entity 5'],
                'Foo': np.random.randn(5).tolist(),
                'Bar': np.random.randn(5).tolist(),
               })
ch = ch.set_index('Entity')
ch = ch[['Foo', 'Bar']] # make sure columns have desired order

markers = [
    dict(color = 'rgb(49, 105, 196)'),
    dict(color = 'rgb(195, 197, 201)')
]

widths = [.2, .4]

opacities = [1, .6]

r.plot.baro(df=ch, title='Horizontal Overlay Bar Chart', orientation='v', xlabel='$', markers=markers, widths=widths, opacities=opacities)

{'data': [{'df':                Foo       Bar
   Entity                      
   Entity 1 -1.790145 -0.325172
   Entity 2  2.327553 -1.421649
   Entity 3  2.129176 -0.280672
   Entity 4 -1.217475 -1.390585
   Entity 5  0.882130 -1.975775, 'orientation': 'v', 'type': 'overlayBar'}],
 'markers': [{'color': 'rgb(49, 105, 196)'}, {'color': 'rgb(195, 197, 201)'}],
 'opacities': [1, 0.6],
 'staticPlot': False,
 'title': 'Horizontal Overlay Bar Chart',
 'widths': [0.2, 0.4],
 'x': {'label': '$'}}

In [6]:
# histogram, note that we don't use the index for x values here because there is no x value other than the series

ch = DataFrame({'Foo': np.random.randn(50).tolist(),
                'Bar': np.random.randn(50).tolist(),
               })

markers = [
    dict(color = 'rgb(49, 105, 196)'),
    dict(color = 'rgb(195, 197, 201)')
]

r.plot.histo(df=ch, title='Histogram by Reporter', xlabel='$', markers=markers)

{'data': [{'df':          Bar       Foo
   0   0.227200 -1.324232
   1   0.690035 -0.242672
   2   0.557168  1.370888
   3   0.114939 -0.624767
   4   0.779130 -0.070523
   5  -0.406012  0.067381
   6  -0.309320  0.751917
   7   1.031120 -0.072318
   8  -0.706812  0.656857
   9  -0.801448  0.195134
   10 -0.155568 -0.069519
   11 -0.296997 -1.695150
   12  0.877009  1.068041
   13 -0.037745  0.752550
   14 -0.545109  1.127007
   15  0.502555 -1.224769
   16 -0.093255  0.366270
   17 -1.536103 -0.404430
   18 -1.293479 -0.805617
   19  0.928508  0.620160
   20 -1.959222 -0.221504
   21  0.442055  0.443310
   22 -1.461033 -0.151759
   23 -0.662825  0.142426
   24  0.099818  1.454062
   25  0.270476 -1.285750
   26 -0.229293 -1.697319
   27  0.753703 -1.187583
   28 -0.113466  0.440965
   29 -1.885941  1.088251
   30  1.019858 -0.681579
   31 -1.352954 -0.283604
   32 -0.231771  1.964161
   33 -0.953529 -1.687674
   34  0.675599 -0.001867
   35  0.090804 -1.114852
   36 -0.651222 -0.54074

In [7]:
# line chart

ch = DataFrame(np.random.randn(20, 3))
ch.columns = ['a', 'b', 'c']
ch.a = range(20)
ch = ch.set_index('a')

lines = [
    dict(width = 7, color = 'rgb(49, 105, 196)'),
    dict(width = 3, color = 'rgb(195, 197, 201)')
]

r.h += htmltag.h4('Line Charts')
r.plot.line(df=ch, title='Line Chart', xlabel='Entity', ylabel='%', lines=lines)

{'data': [{'df':            b         c
   a                     
   0  -1.030605  0.547458
   1  -0.502774 -0.376183
   2  -0.353608  1.123967
   3   0.197732  1.203217
   4  -0.288030 -0.236493
   5   0.956108 -1.195435
   6   1.384704  0.151771
   7   0.600588  0.559886
   8   0.089681  0.094962
   9  -0.795430 -0.701732
   10 -0.980151 -1.058429
   11  0.869351 -0.780134
   12  1.107857 -0.531087
   13  0.477846 -1.239293
   14 -0.154109  0.836569
   15  0.879898  0.326431
   16  0.627864  0.730519
   17 -0.420596 -1.053594
   18  1.020475 -0.043649
   19  0.977670 -1.356818, 'type': 'line'}],
 'lines': [{'color': 'rgb(49, 105, 196)', 'width': 7},
  {'color': 'rgb(195, 197, 201)', 'width': 3}],
 'staticPlot': False,
 'title': 'Line Chart',
 'x': {'label': 'Entity'},
 'y': {'label': '%'}}

In [8]:
# time series

ind = date_range('2017-11-02 9:00', periods=20, freq='T')
ch = DataFrame(data=np.random.randn(20,2), index=ind)
ch.columns = ['a', 'b']

lines = [
    dict(width = 7, color = 'rgb(49, 105, 196)'),
    dict(width = 3, color = 'rgb(195, 197, 201)')
]

# just like a regular line except the index is timestamps
r.plot.time(df=ch, title='Time Series', xlabel='Time', ylabel='%', lines=lines)

{'data': [{'df':                             a         b
   2017-11-02 09:00:00  0.636796  0.985804
   2017-11-02 09:01:00  0.341818 -0.580283
   2017-11-02 09:02:00  0.194213  0.978822
   2017-11-02 09:03:00  1.122730 -1.776593
   2017-11-02 09:04:00  1.370516  0.298101
   2017-11-02 09:05:00  0.739894 -0.461746
   2017-11-02 09:06:00 -0.301907 -0.705399
   2017-11-02 09:07:00  0.566171  2.122549
   2017-11-02 09:08:00 -1.041896 -0.879290
   2017-11-02 09:09:00  1.106895  0.739714
   2017-11-02 09:10:00 -1.133860 -1.155445
   2017-11-02 09:11:00  0.534517 -0.658115
   2017-11-02 09:12:00  0.933981  1.050032
   2017-11-02 09:13:00 -0.292370 -2.497345
   2017-11-02 09:14:00 -0.365240 -1.247807
   2017-11-02 09:15:00 -2.093237  0.344386
   2017-11-02 09:16:00  0.912127  1.340634
   2017-11-02 09:17:00  0.696272  0.498619
   2017-11-02 09:18:00  0.775894  0.051981
   2017-11-02 09:19:00 -0.029622 -0.229957, 'type': 'line'}],
 'lines': [{'color': 'rgb(49, 105, 196)', 'width': 7},
  {'color

In [9]:
# scatter plot

ch = DataFrame(np.random.randn(20, 3))
ch.columns = ['a', 'b', 'c']
ch = ch.set_index('a')

markers = [
    dict(size = 7, color = 'rgb(49, 105, 196)'),
    dict(size = 3, color = 'rgb(195, 197, 201)')
]

layout = dict(font=dict(family='Arial',color='#77797c'),
    xaxis=dict(
        #title='Total Cost (%s)' % basis,
        autorange=True,
        showgrid=False,
        zeroline=True,
        showline=False,
        zerolinecolor='#acadaf',
        autotick=True,
        ticks='outside',
        tickcolor='#acadaf',
        hoverformat='.2f',
        showticklabels=True
    ),
    yaxis=dict(
        #title='vs Arrival Mid (%s)' % basis,
        autorange=True,
        showgrid=False,
        zeroline=True,
        showline=False,
        zerolinecolor='#acadaf',
        linewidth=10,
        autotick=True,
        ticks='outside',
        tickcolor='#acadaf',
        hoverformat='.2f',
        showticklabels=True
    ),
    autosize=False,
    width=500,
    height=500,
    showlegend=False,
    displayModeBar=False)

r.h += htmltag.h4('Scatter Plots')
r.plot.scatter(df=ch, xlabel='alpha', ylabel='beta', markers=markers, layout=layout) #title='Scatter Plot'

{'data': [{'df':                   b         c
   a                            
    1.440569 -0.346128 -0.933763
    1.371192 -2.306229 -1.871073
   -0.216598 -0.270363 -0.798691
    0.604361 -0.573449 -0.360504
   -0.642397 -0.602907  0.650360
   -0.740011  1.518723  0.382181
    1.806248  0.136292  0.587423
    0.111911 -0.576190 -1.041736
    0.926566  0.059695  0.352542
    1.024737  0.676927  1.275667
    1.326214  0.440229  0.304933
    1.002218 -0.840777  2.064582
   -0.078972 -0.712174 -0.826262
   -0.265663  1.208241  0.483426
   -0.764219 -0.961262  1.377738
    0.228424  0.654941  1.720767
    1.077675  1.481060 -0.679873
   -0.948084  0.811142 -0.668915
   -0.491194  0.519406  0.785097
    0.832603  0.894503  0.740023, 'type': 'scatter'}],
 'layout': {'autosize': False,
  'displayModeBar': False,
  'font': {'color': '#77797c', 'family': 'Arial'},
  'height': 500,
  'showlegend': False,
  'width': 500,
  'xaxis': {'autorange': True,
   'autotick': True,
   'hoverformat': '.2

In [10]:
# mixed plot types on one graph

ch1 = DataFrame(np.random.randn(20, 2))
ch1.columns = ['a', 'b1']
ch1.sort_values(by='a', ascending=True, inplace=True)
ch1 = ch1.set_index('a')

ch2 = DataFrame(np.random.randn(20, 2))
ch2.columns = ['a', 'b2']
ch2.sort_values(by='a', ascending=True, inplace=True)
ch2 = ch2.set_index('a')

r.h += htmltag.h4('Mixed Type Plots')
r.plot.multi(dfs=[ch1, ch2], types=['line', 'bar'], title='Mixed Line and Bar Plot', xlabel='x label', ylabel='y label')

{'data': [{'df':                  b1
   a                  
   -1.902754  0.055016
   -1.759408  1.229024
   -1.332187  1.274223
   -1.054086 -0.523043
   -0.879287 -0.033535
   -0.680298  0.252184
   -0.672888 -0.616150
   -0.648207  0.460789
   -0.345092 -2.320297
   -0.283014  1.029104
   -0.228053 -1.203196
   -0.168892 -0.891985
   -0.160651 -0.749832
   -0.104923 -1.155270
   -0.090761  1.095558
    0.456370 -1.209267
    0.688171  0.677636
    0.689390  0.270047
    2.040699  0.892010
    2.361148  0.287851, 'type': 'line'}, {'df':                  b2
   a                  
   -1.780589  0.689588
   -1.206842  0.650608
   -0.902008 -0.856678
   -0.875570 -1.281786
   -0.699120 -0.525539
   -0.662280  0.364318
   -0.623438  0.299339
   -0.181810 -0.989080
   -0.068158 -1.146867
   -0.055327  0.381974
    0.025621 -0.449920
    0.059995  0.727783
    0.096183  0.853505
    0.134118 -1.105609
    0.232472  1.800104
    0.267181 -1.016421
    0.300242  0.785084
    0.609173  0.30893

In [11]:
# different axes

ch1 = DataFrame(np.random.randn(20, 2))
ch1.columns = ['x', 'a']
ch1.x = range(20)
ch1 = ch1.set_index('x')

ch2 = DataFrame(np.random.randn(20, 2))
ch2.columns = ['x', 'b']
ch2.x = range(20)
ch2 = ch2.set_index('x')
ch2.b *= 100

# y2_axis is a list of booleans indicating whether each df should be tied to the y2 axis or not
r.plot.multi(dfs=[ch1, ch2], types=['line', 'line'], title='Lines with Different Y Axes', xlabel='x', ylabel='ya', y2_axis=[False,True], y2label='yb')

{'data': [{'df':            a
   x           
   0   0.015514
   1   0.227636
   2   0.395607
   3  -0.038716
   4  -0.940193
   5   0.332286
   6  -1.437240
   7   0.937480
   8   0.645959
   9   1.279482
   10 -0.461300
   11 -1.297305
   12 -0.010282
   13  2.620091
   14 -0.795166
   15 -0.045892
   16 -0.828333
   17  1.619021
   18  0.050127
   19 -0.280979, 'type': 'line', 'y2': False}, {'df':              b
   x             
   0  -104.684011
   1   -37.848250
   2   -80.725309
   3    12.992667
   4    16.172971
   5   115.583879
   6   -80.784514
   7   -19.913835
   8    83.140600
   9   152.201074
   10  -18.619695
   11 -128.964608
   12    1.264456
   13  -11.063070
   14   -6.927855
   15   85.522276
   16   28.085478
   17  -20.956698
   18   68.381143
   19  -56.824921, 'type': 'line', 'y2': True}],
 'staticPlot': False,
 'title': 'Lines with Different Y Axes',
 'x': {'label': 'x'},
 'y': {'label': 'ya'},
 'y2': {'label': 'yb'}}

In [12]:
# ohlc

ch = DataFrame(np.random.randn(4, 4))
ch.columns = ['open', 'high', 'low', 'close']
ch['high'] = ch['high'] + 3
ch['low'] = ch['low'] - 3

ch['dealer'] = range(4)
ch = ch.set_index('dealer')

r.h += htmltag.h4('Open High Low Close (OHCL) Plot')
r.plot.ohlc(df=ch, title='OHLC Plot by Reporter', xlabel='price', ylabel='instrument')

{'data': [{'df':             open      high       low     close
   dealer                                        
   0      -0.797561  2.727420 -4.125288  1.329024
   1      -0.252218  1.522585 -1.603358  0.067265
   2      -0.111633  4.624546 -3.187984  0.106852
   3      -0.551059  4.356583 -3.827337 -1.049282, 'type': 'ohlc'}],
 'name': '',
 'staticPlot': False,
 'title': 'OHLC Plot by Reporter',
 'x': {'label': 'price'},
 'y': {'label': 'instrument'}}

In [13]:
# pie

ch = DataFrame(data=[1,2,3,4], index=['Factor %s' % x for x in range(4)])

r.h += htmltag.h4('Pie Charts')
r.plot.pie(df=ch, title='Pie')

layout = {'height':300}

# only difference for a donut is the hole argument
r.plot.pie(df=ch, title='Donut', hole=.4, layout=layout)

{'data': [{'df':           0
   Factor 0  1
   Factor 1  2
   Factor 2  3
   Factor 3  4, 'hole': 0.4, 'type': 'pie'}],
 'layout': {'height': 300},
 'staticPlot': False,
 'title': 'Donut'}

In [14]:
# error bar charts

ch = DataFrame(np.random.randn(20, 4))
ch.columns = ['x','y','e1','e2']
ch = ch.set_index('x')
ch['e1'] = ch['e1'].abs()*0.1
ch['e2'] = ch['e2'].abs()*0.1

r.h += htmltag.h4('Error Bar Charts')
# note only first 3 columns are used for symmetric error bars
r.plot.errbar(df=ch, title='Symmetric Error Bars')

# just add the flag symmetric=False, otherwise it defaults to True like in the call above
r.plot.errbar(df=ch, title='Asymmetric Error Bars', symmetric=False)

{'data': [{'df':                   y        e1        e2
   x                                      
   -1.539847  0.505451  0.022329  0.199441
   -0.915269  0.856328  0.054009  0.079488
    0.857270 -0.917841  0.154115  0.105349
   -1.629912  0.746415  0.097020  0.007180
    0.777334  0.475820  0.147720  0.123790
   -1.233879  0.405725  0.005123  0.001949
    1.018666 -0.457004  0.093397  0.042042
    0.480004  0.319967  0.108060  0.019693
    1.199256  0.405840  0.300837  0.068795
    1.597278  0.826564  0.019560  0.039971
    1.349409  0.711527  0.013471  0.128120
   -2.097500  0.430047  0.234076  0.133467
    0.696653 -1.451173  0.048438  0.090663
    0.055198 -0.515392  0.045032  0.025270
   -1.506431 -0.394713  0.156167  0.015130
    0.505373 -0.114542  0.088161  0.012529
    1.395322  1.005502  0.054522  0.011329
   -2.537596  0.433796  0.186063  0.004153
    0.444969  0.148238  0.022931  0.100498
   -0.605533 -0.120643  0.166922  0.139319,
   'errorBars': {'symmetric': False},
 

In [15]:
# error line plots

ch = DataFrame({'day': ['1', '2', '3', '4', '5', '6', '7', '8'],
                'mean': [15, 13, 7, 18, 19, 24, 25, 28],
                'stdev': [1, 2, 1.3, 1, 2.5, .5, 1.5, .75]})
ch = ch[['day', 'mean', 'stdev']] # make sure columns have desired order
ch = ch.set_index('day')

r.h += htmltag.h4('Error Line Plots')
r.plot.errline(df=ch, title='Symmetric Error Bars')

{'data': [{'df':      mean  stdev
   day             
   1      15   1.00
   2      13   2.00
   3       7   1.30
   4      18   1.00
   5      19   2.50
   6      24   0.50
   7      25   1.50
   8      28   0.75,
   'fillcolor': 'rgba(0,100,80,0.2)',
   'type': 'continuousErrorBars'}],
 'staticPlot': False,
 'title': 'Symmetric Error Bars'}

In [16]:
# generate and save the report HTML
r.generate()

saved report to /home/jason/ideal/reports/sample_report/output/sample_plots.html
